In [1]:
import os 
import pandas as pd
import numpy as np
import arcpy


# Enable overwriting of existing files
arcpy.env.overwriteOutput = True

In [2]:
# Copy the pristine data shapefile so that I can add fields and mess with it
input_shapefile = os.path.join("..",  'Data/Parcels/maucotmkshp', 'maucotmk.shp') # The data file to copy
output_shapefile = os.path.join("temp", 'maucotmk_copy.shp')
arcpy.CopyFeatures_management(input_shapefile, output_shapefile)

# Adding and calulating the tax plat key field
shp_path = os.path.join("temp", 'maucotmk_copy.shp')   # specify which shapefile to mess with

arcpy.management.AddField(shp_path, "tax_plat", "TEXT", "", "", "40")  # add the new tax plat field 
arcpy.management.CalculateField(shp_path, "tax_plat", 'str(!TMK!)[0:6]', "PYTHON3")  # Calculates the new plat key 

# pairwise dissolve to dissolve all features with identical tax_plat_keys

<Result 'temp\\maucotmk_copy.shp'>

In [11]:
 #pairwise dissolve to dissolve all features with identical tax_plat_keys



# Experiment down here

dissolve_output = os.path.join("temp", 'taxplat_dissolve.shp')

arcpy.management.Dissolve(shp_path, dissolve_output, ["tax_plat"])

ExecuteError: ERROR 160196: Invalid Topology
Failed to execute (Dissolve).


In [10]:
# Check geometry 
arcpy.CheckGeometry_management(input_shapefile, r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Scripts\geometry_issues.txt")
# Repair geometry 
arcpy.RepairGeometry_management(input_shapefile)




<Result '..\\Data/Parcels/maucotmkshp\\maucotmk.shp'>

In [3]:
# read data from shapefile and make reasonable pandas dataframe that I can display here 

# The data file to read
shp_path = os.path.join("temp", 'maucotmk_copy.shp')

columns_nams = [field.name for field in arcpy.ListFields(shp_path)]     # List of all col names
columns_nams.pop(1)  # remove stupid shape col                           # THe "Shape" col will make numpy array to pandas puke
temparr = arcpy.da.FeatureClassToNumPyArray(shp_path, columns_nams)     # convert to numpy recarray
OSDS_wGrid = pd.DataFrame(temparr)                                       # Convert to pandas bliss
OSDS_wGrid

FID        TMK    TMK_txt   cty_tmk  LandValue  LandExempt  \
0          0  211001001  211001001  11001001        200         200   
1          1  211001002  211001002  11001002       9400        9400   
2          2  211001003  211001003  11001003     124900      124900   
3          3  211001004  211001004  11001004      35000           0   
4          4  211001005  211001005  11001005      16500           0   
...      ...        ...        ...       ...        ...         ...   
51876  51876  261001002  261001002  61001002    2164100     2164100   
51877  51877  261001003  261001003  61001003     171600      171600   
51878  51878  261001004  261001004  61001004     750000      750000   
51879  51879  261001005  261001005  61001005     633000      633000   
51880  51880  261001006  261001006  61001006        100           0   

       BldgValue  BldgExempt  PittCode Homeowner NHoodCode   TaxAcres  \
0              0           0       600         N    1423-5    15.3800   
1              0           0       600         N    1122-6     0.6700   
2              0           0       600         N    1122-6     8.9200   
3              0           0       600         N    1122-6     2.5000   
4              0           0       600         N    1122-6     1.1800   
...          ...         ...       ...       ...       ...        ...   
51876          0           0       999         N    6154-5  7213.7750   
51877          0           0       600         N    5953-6    22.8800   
51878          0           0       600         N    5953-6    50.0000   
51879          0           0       600         N    5953-6    42.2020   
51880          0           0       600         N    5953-6     0.0226   

          GISAcres                                          qpub_link  \
0        24.718663  https://qpublic.schneidercorp.com/Application....   
1         1.412801  https://qpublic.schneidercorp.com/Application....   
2         8.412005  https://qpublic.schneidercorp.com/Application....   
3         2.499308  https://qpublic.schneidercorp.com/Application....   
4         1.174124  https://qpublic.schneidercorp.com/Application....   
...            ...                                                ...   
51876  7113.443641  https://qpublic.schneidercorp.com/Application....   
51877    22.643961  https://qpublic.schneidercorp.com/Application....   
51878    57.755559  https://qpublic.schneidercorp.com/Application....   
51879    42.497785  https://qpublic.schneidercorp.com/Application....   
51880     0.144956  https://qpublic.schneidercorp.com/Application....   

         Shape_Leng    Shape_Area tax_plat  
0       2937.632735  1.000329e+05   211001  
1        338.916826  5.717401e+03   211001  
2       1560.773608  3.404218e+04   211001  
3        420.385964  1.011434e+04   211001  
4        291.173935  4.751513e+03   211001  
...             ...           ...      ...  
51876  47307.328495  2.878709e+07   261001  
51877   1242.587340  9.163686e+04   261001  
51878   2469.117529  2.337285e+05   261001  
51879   2552.565023  1.719824e+05   261001  
51880     97.427171  5.866158e+02   261001  

[51881 rows x 17 columns]